In [25]:
class ChessPiece:
    def __init__(self, color, symbol):
        """
        Initialize a chess piece with the given color and symbol.

        Args:
            color (str): The color of the chess piece (e.g., 'white' or 'black').
            symbol (str): The symbol representing the piece (e.g., '♚' for black king).

        Attributes:
            color (str): The color of the chess piece.
            symbol (str): The symbol representing the piece.
        """
        self.color = color
        self.symbol = symbol

In [26]:
class King(ChessPiece):
    """
    A class representing the king chess piece.

    Attributes:
        counter (int): Keeps track of the number of movements for the king (used for castling).
        check_board (CheckBoard): An object from the CheckBoard class to detect check conditions.
    """

    def __init__(self, color, symbol):
        super().__init__(color, symbol)
        self.counter = 0

    def move(self, board, start, end):
        """
        Validate whether a move is acceptable for the king.

        Args:
            board (list): The chessboard.
            start (tuple): The starting position (row, column).
            end (tuple): The ending position (row, column).

        Returns:
            bool: True if the move is valid, False otherwise.
        """
        # Check conditions for castling
        if (
            self.counter == 0
            and abs(start[1] - end[1]) == 2
            and start[0] == end[0]
        ):
            return True
        # Check conditions for normal movement
        elif (
                (abs(start[0] - end[0]) == 1 and start[1] == end[1])
                or (abs(start[1] - end[1]) == 1 and start[0] == end[0])
                or (abs(start[0] - end[0]) == 1 and abs(start[1] - end[1]) == 1)
        ):
            return True
        else:
            return False 

In [27]:
class Queen(ChessPiece):
    """
    A class representing the queen chess piece.

    Attributes:
        None

    Methods: 
        validate_move
        move
    """

    def validate_move(self, start, end):
        """
        Validate whether a move is acceptable for the queen based on the queen movement logic 

        Args:
            start (tuple): The starting position (row, column).
            end (tuple): The ending position (row, column).

        Returns:
            bool: True if the move is valid, False otherwise.
        """
        if abs(end[0] - start[0]) == abs(end[1] - start[1]):
            return True
        elif start[1] == end[1]:
            return True
        elif start[0] == end[0]:
            return True
        else:
            return False

    def move(self, board, start, end):
        """
        Check the path of move to be free of pieces

        Args:
            board (list): The chessboard.
            start (tuple): The starting position (row, column).
            end (tuple): The ending position (row, column).

        Returns:
            bool: True if the move is possible, False otherwise.
        """
        if not self.validate_move(start, end):
            return False

        # Check path for obstacles
        step_row = -1 if start[0] > end[0] else 1 if start[0] < end[0] else 0
        step_col = -1 if start[1] > end[1] else 1 if start[1] < end[1] else 0

        row, col = start[0] + step_row, start[1] + step_col
        while row != end[0] or col != end[1]:
            if board[row][col]:
                return False

            row += step_row
            col += step_col
        return True


In [28]:
class Rook(ChessPiece):
    """
    A class representing the rook chess piece.

    Attributes:
        counter (int): Keeps track of the number of movements for the rook (used for castling).
        promote (bool): Indicates whether the rook was promoted from a pawn (affects castling).
    """

    def __init__(self, color, symbol):
        super().__init__(color, symbol)
        #To count the number of movement for rook. we need this attribute for castling
        self.counter = 0
        #this attribute is used for castling. if the rook was a pawn promoted, we can't do castling
        self.promote = False   

    """
    Check whether the path between start and end positions is clear for the rook.

    Args:
        board (list): The chessboard.
        start (tuple): The starting position (row, column).
        end (tuple): The ending position (row, column).

    Returns:
        bool: True if the path is clear, False if there are obstacles.
    """
    
    def check_path(self, board, start, end):
        max_col = max (start[1], end[1])
        max_row = max (start[0], end[0])
        min_col = min (start[1], end[1])
        min_row = min (start[0], end[0])
        if start[0] == end[0]:
            for cell in range(min_col + 1, max_col):
                if board[start[0]][cell]:
                    return False
            else:
                return True
        elif start[1] == end[1]:
            for cell in range(min_row + 1, max_row):
                if board[cell][start[1]]:
                    return False
            else:
                return True

    def move(self, board, start, end):
        
        if (start[0] == end[0] or start[1] == end[1]) and self.check_path(board, start, end):
            return True
        else:
            return False 

In [29]:
class Bishop(ChessPiece):
    """
    A class representing the bishop chess piece.

    Attributes:
        None

    Methods:
        check_path 
        move    
    """
   # Check if the path is empty
    def check_path(self, board, start, end):
        dr = end[0] - start[0]
        dc = end[1] - start[1]

        # Determining the direction of vertical movement        
        if dr > 0: r_sign = 1 
        else: r_sign = -1

        # Determining the direction of horizontal movement
        if dc > 0: c_sign = 1 
        else: c_sign = -1 

        # Checking that all the squares in line with the destination square are empty
        for step in range(1, abs(dr)):
            row = start[0] + step * r_sign
            col = start[1] + step * c_sign
            if board[row][col]:
                return False
        return True
    
    def move(self, board, start, end):

        # Movement is allowed if the row and column distance of the origin and destination squares are equal 
        if abs(end[0] - start[0]) != abs(end[1] - start[1]):
            return False
        
        return self.check_path(board, start, end)

In [30]:
class Knight(ChessPiece):
    """ 
        A class representing the Knight chess piece.
        Methods:
            move
    """
    # Knight's movement logic
    def move(self, board, start, end):
        if (abs(start[1] - end[1]) == 1 and abs(start[0] - end[0]) == 2) or (abs(start[0] - end[0]) == 1 and abs(start[1] - end[1]) == 2):
            return True
        else:
            return False   

In [31]:
class Pawn(ChessPiece):

    # Pawn's movement logic
    def move(self, board, start, end, EnPassant_previous_move):
        capture = False     # Specifying the move along with capturing the opponent's piece to send to the move_piece method
        EnPassant = False

    # Move one square forward
        if self.color == "white":
            dir = -1        # The white pawn moves upwards
            start_row = 6   # The white pawn first location
        else:
            dir = 1         # The black pawn moves downwards
            start_row = 1   # The black pawn first location

        dr = end[0] - start[0] 
        dc = abs(end[1] - start[1])

        # Checking the permitted direction of movement and the difference between the origin and destination columns and rows
        if dr * dir < 0 or abs(dr) > 2 or dc > 1:
            return False, capture, EnPassant
        if dc == 1:
            if abs(dr) == 0:
                return False, capture, EnPassant 

        # Check if the destination square is not empty and the movement is not capturing
        if board[end[0]][end[1]] and dc == 0:
            return False, capture, EnPassant

    # Move two squares forward
        if dr == 2 * dir:
            # This movement is valid only in first move
            if start[0] == start_row:
                # check if it is the first move and the emptiness of the middle square
                if dc != 0 or board[start[0] + dir][start[1]]:
                    return False, capture, EnPassant
            else:
                return False, capture, EnPassant  

    # Diagonal movement along with capturing the opponent's piece (EnPassant or usual)
        if dc == 1 and dir == dr:
            capture = True

            # Is the destination square empty of any pieces?
            if board[end[0]][end[1]] is None:

                # Check for EnPassant possibility
                if EnPassant_previous_move[0]:
                    previous_pawn_row, previous_pawn_col = EnPassant_previous_move[1]

                    if (
                        board[previous_pawn_row][previous_pawn_col].color != self.color
                        and (
                            (board[previous_pawn_row][previous_pawn_col].color == "white" and previous_pawn_row == 4)
                            or (board[previous_pawn_row][previous_pawn_col].color == "black" and previous_pawn_row == 3)
                        )
                        and abs(start[1] - previous_pawn_col) == 1
                        and start[0] == previous_pawn_row
                        and end[1] == previous_pawn_col
                    ):
                        EnPassant = True
                    else:
                        return False, capture, EnPassant                       

                else:
                    return False, capture, EnPassant
                
            # Is the piece located in the destination square the opponent's piece?    
            elif board[end[0]][end[1]].color == self.color:
                return False, capture, EnPassant               

        # All conditions are met, so movement is valid
        return True, capture, EnPassant

In [32]:
import copy

class CheckBoard:
    # to check the destination square is empty or not
    def empty_square(self, board, position):
        if not board[position[0]][position[1]]:
            return True
        return False
            
    def capture(self, board, piece_color, position):
        """
        check an opponent's piece can be captured in current move or not 
        """
        if board[position[0]][position[1]].color != piece_color and not isinstance(board[position[0]][position[1]], King):
            return True
        return False
    

    def check(self, board, king_position, piece_color):
        """A method to check whether the king is check by opponent's pieces"""
        for row in range(8):
            for col in range(8):
                if isinstance(board[row][col], Pawn):
                    if board[row][col] and board[row][col].move(board, (row,col), king_position, [False, [0,0]])[0] and board[row][col].color != piece_color:
                        return True
                elif not isinstance(board[row][col], King):
                    if board[row][col] and board[row][col].move(board, (row,col), king_position) and board[row][col].color != piece_color:
                        return True
        
        return False
            
    def checkmate(self, board, king_position, piece_color):
        """A method to check whether the king is checkmate by opponent's pieces"""
        if self.check(board, king_position, piece_color):
            possible_position_for_king = []
            # Define possible movements for the king
            king_moves = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (-1, -1), (1, -1), (-1, 1)]
            
            # Iterate through possible movements
            for move in king_moves:
                # Calculate new position
                new_position = (king_position[0] + move[0], king_position[1] + move[1])
                
                # Check if new position is within the board boundaries
                if 0 <= new_position[0] < 8 and 0 <= new_position[1] < 8:
                    # Check if the new position is empty or occupied by an opponent's piece
                    if ((
                        board[new_position[0]][new_position[1]] == None 
                        or board[new_position[0]][new_position[1]].color != piece_color
                        ) 
                        and not self.check(board, new_position, piece_color)
                    ):
                        possible_position_for_king.append(new_position)
            
            if len(possible_position_for_king) > 0:
                return True
            elif len(possible_position_for_king) == 0:    
                # If the king couldn't make a move, come and check if there is a piece to fix the king.
                for row in range(8):
                    for col in range(8):
                        if board[row][col] and board[row][col].color == piece_color and not isinstance(board[row][col], King):
    
                            for row2 in range(8):
                                for col2 in range(8):
                                    if isinstance(board[row][col], Pawn):
                                        if (
                                            not isinstance(board[row2][col2], King)
                                            and board[row][col].move(board, (row,col), (row2,col2), [False, [0, 0]])[0]
                                            and (self.empty_square(board, (row2,col2)) or self.capture(board,piece_color, (row2,col2)))
                                        ):
                                            copy_of_board = copy.deepcopy(board)
                                            copy_of_board[row2][col2] = copy_of_board[row][col]
                                            copy_of_board[row][col] = None
                                            if not self.check(copy_of_board, king_position,piece_color):
                                                return True
                                    else:
                                        if (
                                            not isinstance(board[row2][col2], King)
                                            and board[row][col].move(board, (row,col), (row2,col2))
                                            and (self.empty_square(board, (row2,col2)) or self.capture(board,piece_color, (row2,col2)))
                                        ):
                                            copy_of_board = copy.deepcopy(board)
                                            copy_of_board[row2][col2] = copy_of_board[row][col]
                                            copy_of_board[row][col] = None
                                            if not self.check(copy_of_board, king_position,piece_color):
                                                return True

                return False
        else:
            return True


    def insufficient_material(self, board):
        """It is a method that measures the lack of enough force for matting.
          In this case, two players do not have enough power to checkmate the chess game. This happens in three situations:
            1- The king against the king
            2- The king and the knight against the king alone
            3- The king and the bishop against the king alone
        """
        white_pieces = []
        black_pieces = []
        # we want to count the number of white pieces and black pieces
        for row in range(8):
            for col in range(8):
                if board[row][col]:
                    if board[row][col].color == "white":
                        white_pieces.append(board[row][col])
                    elif board[row][col].color == "black":
                        black_pieces.append(board[row][col])
         # Check if there are only kings or one side has only king and the other side has king and one minor piece
        if len(white_pieces) == 1 and len(black_pieces) == 1:
            return True
        elif len(white_pieces) == 1 and len(black_pieces) == 2 and any(isinstance(piece, (Bishop, Knight)) for piece in black_pieces):
            return True
        elif len(black_pieces) == 1 and len(white_pieces) == 2 and any(isinstance(piece, (Bishop, Knight)) for piece in white_pieces):
            return True
        elif len(black_pieces) == 2 and len(white_pieces) == 2 and any(isinstance(piece, Bishop) for piece in white_pieces) and any(isinstance(piece, Bishop) for piece in black_pieces):
            return True
        return False

      
    def stalemate(self, board, king_position, piece_color):#logic
        """A stalemate is a position in which the player whose turn it is to move
        has no legal move and his king is not in check. 
        A stalemate results in an immediate draw."""
        if not self.check(board, king_position,piece_color):
            possible_position_for_king = []
            # Define possible movements for the king
            king_moves = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (-1, -1), (1, -1), (-1, 1)]
            
            # Iterate through possible movements
            for move in king_moves:
                # Calculate new position
                new_position = (king_position[0] + move[0], king_position[1] + move[1])
                
                # Check if new position is within the board boundaries
                if 0 <= new_position[0] < 8 and 0 <= new_position[1] < 8:
                    # Check if the new position is empty or occupied by an opponent's piece
                    if ((
                        board[new_position[0]][new_position[1]] == None 
                        or board[new_position[0]][new_position[1]].color != piece_color
                        ) 
                        and not self.check(board, new_position, piece_color)
                    ):
                        possible_position_for_king.append(new_position)

            if len(possible_position_for_king) == 0:            # TO check that the king has choice or not
                # To check that the player has legal move for other pieces or not
                
                for row in range(8):
                    for col in range(8):
                        if board[row][col] and board[row][col].color == piece_color and not isinstance(board[row][col], King):
    
                            for row2 in range(8):
                                for col2 in range(8):
                                    if isinstance(board[row][col], Pawn):
                                        if (
                                            not isinstance(board[row2][col2], King)
                                            and board[row][col].move(board, (row,col), (row2,col2), [False, [0, 0]])[0]
                                            and (self.empty_square(board, (row2,col2)) or self.capture(board,piece_color, (row2,col2)))
                                        ):
                                            copy_of_board = copy.deepcopy(board)
                                            copy_of_board[row2][col2] = copy_of_board[row][col]
                                            copy_of_board[row][col] = None
                                            if not self.check(copy_of_board, king_position,piece_color):
                                                return False #continue the game
                                    else:
                                        if (
                                            not isinstance(board[row2][col2], King)
                                            and board[row][col].move(board, (row,col), (row2,col2))
                                            and (self.empty_square(board, (row2,col2)) or self.capture(board,piece_color, (row2,col2)))
                                        ):
                                            copy_of_board = copy.deepcopy(board)
                                            copy_of_board[row2][col2] = copy_of_board[row][col]
                                            copy_of_board[row][col] = None
                                            if not self.check(copy_of_board, king_position,piece_color):
                                                return False  #continue the game
                            
                return True    # game ends without victory for either player
            else:
                return False #continue the game
        else:
            return False #continue the game   
    
    def five_same_moves(self, moves):
        if moves.count(moves[-1]) == 5 :
            return True
        return False
    
    def fifty_moves(self, fifty_moves_count):
        if fifty_moves_count == 50:
            return True
        return False    

    #to check after this move the king will not be in check and 
    #also it's not in check already and to check is the game ended (checkmate or draw occured) or not
    def valid_movement(self, board, start, end, piece_color):
        copy_of_board = copy.deepcopy(board)
        copy_of_board[end[0]][end[1]] = copy_of_board[start[0]][start[1]]
        copy_of_board[start[0]][start[1]] = None
        for row in range(8):
            for col in range(8):
                if (
                    isinstance(copy_of_board[row][col], King)
                    and copy_of_board[row][col].color == piece_color
                    and self.check(copy_of_board, (row, col), piece_color)
                ):
                    return False   # if after the movement King is in check return False
        return True  # if after the movement King isn't in check return True 

In [39]:
class ChessBoard:
    def __init__(self):
        self.board = [[None for _ in range(8)] for _ in range(8)]
        # storing captured pieces
        self.white_captured_pieces = []
        self.black_captured_pieces = []

        # storing the EnPassant possibility and the position of the target pawn 
        self.EnPassant_previous_move = [False, [0, 0]]

        # Saving the moves to check that if there are five the same, a draw occurs
        # *** It has to be actually three same moves to declare a draw but in online games a draw is called after five repetitions
        self.moves = []

        # for counting consecutive moves where neither player has moved a pawn or captured a piece
        self.fifty_moves_count = 0

        # Place pieces on the board at first
        self.place_pieces()

        self.check_board = CheckBoard()


    # Place pieces on the board at first
    def place_pieces(self):
        self.board[7][0] = Rook("white", "♖")
        self.board[7][1] = Knight("white", "♘")
        self.board[7][2] = Bishop("white", "♗")
        self.board[7][3] = Queen("white", "♕")
        self.board[7][4] = King("white", "♔")
        self.board[7][5] = Bishop("white", "♗")
        self.board[7][6] = Knight("white", "♘")
        self.board[7][7] = Rook("white", "♖")
        for i in range(8):
            self.board[6][i] = Pawn("white", "♙")

        self.board[0][0] = Rook("black", "♜")
        self.board[0][1] = Knight("black", "♞")
        self.board[0][2] = Bishop("black", "♝")
        self.board[0][3] = Queen("black", "♛")
        self.board[0][4] = King("black", "♚")
        self.board[0][5] = Bishop("black", "♝")
        self.board[0][6] = Knight("black", "♞")
        self.board[0][7] = Rook("black", "♜")
        for i in range(8):
            self.board[1][i] = Pawn("black", "♟")

    # Move a piece on the board
    def move_any_pieces(self, start, end):
        piece = self.board[start[0]][start[1]]
        # if the piece is a pawn
        if isinstance(piece, Pawn):
            legal_move, capture, EnPassant = piece.move(self.board, start, end, self.EnPassant_previous_move)
            if legal_move and self.check_board.valid_movement(self.board, start, end, piece.color):
                # Reset the EnPassant boolean to be ready to update in the move_pawn method
                self.EnPassant_previous_move[0] = False
                # We are looking for fifty consecutive moves where neither player has moved a pawn or captured a piece to declare a draw, and every pawn move breaks this sequence
                self.fifty_moves_count = 0
                # store the move symbol with the end position as a str to use in five_same_moves method in CheckBoard class
                self.moves.append(self.board[start[0]][start[1]].symbol + str(end[0]) + str(end[1]))
                self.move_pawn(piece, start, end, capture, EnPassant)
                return True
            else:
                return False
            

        elif isinstance(piece, King):  # if the piece is king
            legal_move = piece.move(self.board, start, end)
            if legal_move and self.check_board.valid_movement(self.board, start, end, piece.color):
                # store the move symbol with the end position as a str to use in five_same_moves method in the CheckBoard class
                self.moves.append(self.board[start[0]][start[1]].symbol + str(end[0]) + str(end[1]))

                self.move_king(piece, start, end)
                piece.counter += 1
                return True
            else:
                return False
            
        elif isinstance(piece, Rook):
            legal_move = piece.move(self.board, start, end)
            if legal_move and self.check_board.valid_movement(self.board, start, end, piece.color):
                self.move_some_pieces(piece, start, end)
                piece.counter += 1
                return True
            else:
                return False 

        else: # other pieces 
            legal_move = piece.move(self.board, start, end)
            if legal_move and self.check_board.valid_movement(self.board, start, end, piece.color):
                if self.move_some_pieces(piece, start, end):
                    return True
                else:
                    return False
            else:
                return False
    
    def move_pawn(self, piece, start, end, capture, EnPassant):

        if end[0] == 7 and piece.color == "black":
            self.promote_pawn(start, "black", ['♛', '♜', '♞', '♝'])
        elif end[0] == 0 and piece.color == "white":
            self.promote_pawn(start, "white", ['♕', '♖', '♘', '♗'])
            
        if capture:
            if EnPassant:
                if piece.color == "white":
                    self.black_captured_pieces.append(self.board[self.EnPassant_previous_move[1][0]][self.EnPassant_previous_move[1][1]].symbol) 
                else:
                    self.white_captured_pieces.append(self.board[self.EnPassant_previous_move[1][0]][self.EnPassant_previous_move[1][1]].symbol)    
                self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
                self.board[start[0]][start[1]] = None
                self.board[self.EnPassant_previous_move[1][0]][self.EnPassant_previous_move[1][1]] = None  
            else:   
                if piece.color == "white": 
                    self.black_captured_pieces.append(self.board[end[0]][end[1]].symbol) 
                else:
                    self.white_captured_pieces.append(self.board[end[0]][end[1]].symbol)    
                self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
                self.board[start[0]][start[1]] = None    
        else:
            self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
            self.board[start[0]][start[1]] = None
            # The else block runs when the move is not a capture one. Then, if it is a two-square move, we can have En Passant in the next move with some conditions that will checked in Pawn's class.
            if abs(end[0] - start[0]) == 2: 
                self.EnPassant_previous_move[0] = True
                self.EnPassant_previous_move[1] = end 

    def move_some_pieces(self, piece, start, end):

        # store the move symbol with the end position as a str to use in five_same_moves method in the CheckBoard class
        self.moves.append(self.board[start[0]][start[1]].symbol + str(end[0]) + str(end[1]))

        if self.check_board.empty_square(self.board, end):
            self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
            self.board[start[0]][start[1]] = None

            # We are looking for fifty consecutive moves where neither player has moved a pawn or captured a piece to declare a draw and this move is counting in 
            self.fifty_moves_count += 1
            return True

        elif self.check_board.capture(self.board, piece.color, end):  
            if piece.color == "white":
                self.black_captured_pieces.append(self.board[end[0]][end[1]].symbol)
            else:
                self.white_captured_pieces.append(self.board[end[0]][end[1]].symbol)    
            self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
            self.board[start[0]][start[1]] = None 

            # We are looking for fifty consecutive moves where neither player has moved a pawn or captured a piece to declare a draw, and every capture breaks this sequence
            self.fifty_moves_count = 0
            return True
        else:
            return False

    def promote_pawn(self, start, color, symbols):
        new_piece = "new_piece"
        while new_piece.lower() not in ["queen", "rook", "knight", "bishop"]:
            new_piece = input(
                "Which piece would you like your pawn to become: queen, rook, knight, or bishop?"
            )
        if new_piece.lower() == "queen":
            self.board[start[0]][start[1]] = Queen(color, symbols[0])
        elif new_piece.lower() == "rook":
            self.board[start[0]][start[1]] = Rook(color, symbols[1])
            self.board[start[0]][start[1]].promote = True  # if the pawn promote to rook, we can't do castling
        elif new_piece.lower() == "knight":
            self.board[start[0]][start[1]] = Knight(color, symbols[2])
        else:
            self.board[start[0]][start[1]] = Bishop(color, symbols[3])

    # This function specifies that if the king's move is a castle,
    # it will determine the location of the king and rook, and if it is a normal move, it will return it.
    def move_king(self, piece, start, end):
        # First, we have to make sure that two kings are not together
        for row in range(8):
            for col in range(8):
                if (
                    isinstance(self.board[row][col], King)
                    and self.board[row][col].color != piece.color
                    and abs(end[0] - row) <= 1
                    and abs(end[1] - col) <= 1
                ):
                    return False 
                
        # Castling checks
        if abs(start[1] - end[1]) == 2:
            if (
                (start[1] - end[1]) < 0
                and self.check_board.empty_square(self.board, end)
                and self.check_board.empty_square(self.board, (end[0], min(start[1], end[1]) + 1))
                and not self.check_board.check(self.board, end, piece.color)
                and not self.check_board.check(self.board, (end[0], min(start[1], end[1]) + 1), piece.color)
                ):

             # if player wants to do castling with the right rook
                if (
                    piece.color == "white"
                    and isinstance(self.board[7][7], Rook)
                    and self.board[7][7].counter == 0
                    and not self.board[7][7].promote
                ):
                    self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
                    self.board[start[0]][start[1]] = None
                    self.board[end[0]][end[1] - 1] = self.board[7][7]
                    self.board[7][7] = None
                    self.fifty_moves_count += 1
                    return True

                elif (
                    piece.color == "black"
                    and isinstance(self.board[0][7], Rook)
                    and self.board[0][7].counter == 0
                    and not self.board[0][7].promote
                ):
                    self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
                    self.board[start[0]][start[1]] = None
                    self.board[end[0]][end[1] - 1] = self.board[0][7]
                    self.board[0][7] = None
                    self.fifty_moves_count += 1
                    return True
            elif (
                (start[1] - end[1]) > 0
                and self.check_board.empty_square(self.board, end)
                and self.check_board.empty_square(self.board, (end[0], min(start[1], end[1]) + 1))
                and not self.check_board.check(self.board, end, piece.color)
                and not self.check_board.check(self.board, (end[0], min(start[1], end[1]) + 1), piece.color)
                ):  # if player wants to do castling with the left rook
                if (
                    piece.color == "white"
                    and isinstance(self.board[7][0], Rook)
                    and self.board[7][0].counter == 0
                    and not self.board[7][0].promote
                ):
                    self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
                    self.board[start[0]][start[1]] = None
                    self.board[end[0]][end[1] + 1] = self.board[7][0]
                    self.board[7][0] = None
                    self.fifty_moves_count += 1
                    return True

                elif (
                    piece.color == "black"
                    and isinstance(self.board[0][0], Rook)
                    and self.board[0][0].counter == 0
                    and not self.board[7][0].promote
                ):
                    self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
                    self.board[start[0]][start[1]] = None
                    self.board[end[0]][end[1] + 1] = self.board[0][0]
                    self.board[0][0] = None
                    self.fifty_moves_count += 1
                    return True

        # Normal moves and captures
        else:
            if self.check_board.empty_square(self.board, end) and not self.check_board.check(self.board, end, piece.color):
                self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
                self.board[start[0]][start[1]] = None

                # We are looking for fifty consecutive moves where neither player has moved a pawn or captured a piece to declare a draw and this move is counting in 
                self.fifty_moves_count += 1
                return True

            elif self.check_board.capture(self.board, piece.color, end) and not self.check_board.check(self.board, end, piece.color):  
                if piece.color == "white":
                    self.black_captured_pieces.append(self.board[end[0]][end[1]].symbol)
                else:
                    self.white_captured_pieces.append(self.board[end[0]][end[1]].symbol)    
                self.board[end[0]][end[1]] = self.board[start[0]][start[1]]
                self.board[start[0]][start[1]] = None 

                # We are looking for fifty consecutive moves where neither player has moved a pawn or captured a piece to declare a draw, and every capture breaks this sequence
                self.fifty_moves_count = 0
                return True
            else:
                return False


    def find_king(self, piece_color):
        for row in range(8):  
            for col in range(8):
                if isinstance(self.board[row][col], King) and self.board[row][col].color == piece_color:
                    return row, col    
                 

    def is_draw(self, piece_color):
        if (
            self.check_board.fifty_moves(self.fifty_moves_count) 
            or self.check_board.five_same_moves(self.moves)
            or self.check_board.stalemate(self.board, self.find_king(piece_color), piece_color)
            or self.check_board.insufficient_material(self.board)
        ):
            return True
        else:
            return False


    def convert_input(self):
    
        # to convert column letter to index
        col_map = {'a': 0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7}
        
        # get the valid input
        while True:
            user_input = input("Enter the movement. For example, your input can be like \"a7 a5\" to move the left-most white pawn in the first movement ")
            posistions = user_input.lower().split()
            if (
                len(posistions) == len(posistions[0]) == len(posistions[1]) == 2 
                and posistions[0][0] in "abcdefgh" and posistions[0][1] in "12345678" 
                and posistions[1][0] in "abcdefgh" and posistions[1][1] in "12345678"
                and posistions[0] != posistions[1]
            ):
                posistions[0] = posistions[0].strip()
                posistions[1] = posistions[1].strip()
                # convert input to board array indices
                start = (int(posistions[0][1]) - 1, col_map[posistions[0][0]])
                end = (int(posistions[1][1]) - 1, col_map[posistions[1][0]])
                if self.board[start[0]][start[1]]:
                    break
            
        return start, end
    
    
    # Display the current state of the board
    def display(self):
        if self.black_captured_pieces:
            print("captured pieces: ", self.black_captured_pieces)
        print("   -----------------------------------------")
        for row in range(8):
            print(row + 1, " ", end="|")
            for col in range(8):
                if self.board[row][col]:
                    print("", self.board[row][col].symbol, "|", end="")
                else:
                    print("   ", "|", end="")    
            print("\n   -----------------------------------------")
        print("     a    b    c    d    e    f    g    h  ")
        if self.white_captured_pieces:
            print("captured pieces: ", self.white_captured_pieces, '\n')

In [40]:
from IPython.display import clear_output

chess_board = ChessBoard()
chess_board.place_pieces()
chess_board.display()

colors = ["white", "black"]
turn_index = 0
while True: 
    turn = colors[turn_index]
    print(f"the turn is for {turn}")
    start, end = chess_board.convert_input()
    if chess_board.board[start[0]][start[1]].color != turn:
        clear_output()
        chess_board.display()
        print(f"that's not {chess_board.board[start[0]][start[1]].color}'s turn")
    else:
        if chess_board.move_any_pieces(start, end):
            clear_output()
            chess_board.display()
            white_row, white_col = chess_board.find_king("white")
            black_row, black_col = chess_board.find_king("black")
            if not chess_board.check_board.checkmate(chess_board.board, (white_row, white_col), "white"):
                print("The white is checkmated")
                print("The black player has won")
                break
            elif not chess_board.check_board.checkmate(chess_board.board, (black_row, black_col), "black"):
                print("The black is checkmated")
                print("The white player has won")
                break
            elif chess_board.is_draw("white") or chess_board.is_draw("black"):
                print("A draw is happened")
                break

            turn_index = (turn_index + 1) % 2 
        else:
            clear_output()
            chess_board.display()
            print("Invalid Movement")    


   -----------------------------------------
1  |    |    |    | ♜ | ♚ | ♜ |    |    |
   -----------------------------------------
2  |    |    |    | ♟ |    | ♟ |    |    |
   -----------------------------------------
3  |    |    |    |    |    |    |    |    |
   -----------------------------------------
4  |    |    |    |    | ♕ |    |    |    |
   -----------------------------------------
5  |    |    |    |    |    |    |    |    |
   -----------------------------------------
6  |    |    |    |    |    |    |    |    |
   -----------------------------------------
7  |    |    |    |    |    |    |    |    |
   -----------------------------------------
8  |    |    |    |    | ♔ |    |    |    |
   -----------------------------------------
     a    b    c    d    e    f    g    h  
The black is checkmate
The white player has won


## The reference of chess rules applied in this project is:

### https://www.chess.com/learn-how-to-play-chess

In [ ]:
chess_board.move_any_pieces((0,1), (2,2))
chess_board.move_any_pieces((1,0), (3,0))
chess_board.move_any_pieces((0,0), (2,0))
chess_board.move_any_pieces((2,0), (2,1))
chess_board.move_any_pieces((1,3), (3,3))
chess_board.move_any_pieces((0,2), (3,5))
chess_board.move_any_pieces((0,3), (2,3))
chess_board.move_any_pieces((2,3), (2,5))
chess_board.move_any_pieces((2,5), (4,7))
chess_board.move_any_pieces((0,4), (0,3))
chess_board.move_any_pieces((0,3), (1,3))
chess_board.move_any_pieces((1,3), (0,2))
chess_board.move_any_pieces((3,0), (5,0))

chess_board.display()

   -----------------------------------------
1  |    |    | ♚ |    |    | ♝ | ♞ | ♜ |
   -----------------------------------------
2  |    | ♟ | ♟ |    | ♟ | ♟ | ♟ | ♟ |
   -----------------------------------------
3  |    | ♜ | ♞ |    |    |    |    |    |
   -----------------------------------------
4  | ♟ |    |    | ♟ |    | ♝ |    |    |
   -----------------------------------------
5  |    |    |    |    |    |    |    | ♛ |
   -----------------------------------------
6  |    |    |    |    |    |    |    | ♙ |
   -----------------------------------------
7  | ♙ | ♙ | ♙ | ♙ | ♙ | ♙ | ♙ |    |
   -----------------------------------------
8  | ♖ | ♘ | ♗ | ♕ | ♔ | ♗ | ♘ | ♖ |
   -----------------------------------------
     a    b    c    d    e    f    g    h  


In [ ]:
chess_board.move_any_pieces((6,0), (5,1))
chess_board.move_any_pieces((2,1), (5,1))
chess_board.move_any_pieces((6,0), (5,1))
chess_board.move_any_pieces((4,7), (5,7))

chess_board.display()

captured pieces:  ['♜']
   -----------------------------------------
1  |    |    | ♚ |    |    | ♝ | ♞ | ♜ |
   -----------------------------------------
2  |    | ♟ | ♟ |    | ♟ | ♟ | ♟ | ♟ |
   -----------------------------------------
3  |    |    | ♞ |    |    |    |    |    |
   -----------------------------------------
4  | ♟ |    |    | ♟ |    | ♝ |    |    |
   -----------------------------------------
5  |    |    |    |    |    |    |    |    |
   -----------------------------------------
6  |    | ♙ |    |    |    |    |    | ♛ |
   -----------------------------------------
7  |    | ♙ | ♙ | ♙ | ♙ | ♙ | ♙ |    |
   -----------------------------------------
8  | ♖ | ♘ | ♗ | ♕ | ♔ | ♗ | ♘ | ♖ |
   -----------------------------------------
     a    b    c    d    e    f    g    h  
captured pieces:  ['♙'] 



In [ ]:
chess_board.move_any_pieces((7,1), (5,2))
chess_board.display()

captured pieces:  ['♜']
   -----------------------------------------
1  |    |    | ♚ |    |    | ♝ | ♞ | ♜ |
   -----------------------------------------
2  |    | ♟ | ♟ |    | ♟ | ♟ | ♟ | ♟ |
   -----------------------------------------
3  |    |    | ♞ |    |    |    |    |    |
   -----------------------------------------
4  | ♟ |    |    | ♟ |    | ♝ |    |    |
   -----------------------------------------
5  |    |    |    |    |    |    |    | ♛ |
   -----------------------------------------
6  |    | ♙ | ♘ |    |    |    |    | ♙ |
   -----------------------------------------
7  |    | ♙ | ♙ | ♙ | ♙ | ♙ | ♙ |    |
   -----------------------------------------
8  | ♖ |    | ♗ | ♕ | ♔ | ♗ | ♘ | ♖ |
   -----------------------------------------
     a    b    c    d    e    f    g    h  


In [ ]:
chess_board.move_any_pieces((6,1), (7,1))
chess_board.display()

captured pieces:  ['♜']
   -----------------------------------------
1  |    |    | ♚ |    |    | ♝ | ♞ | ♜ |
   -----------------------------------------
2  |    | ♟ | ♟ |    | ♟ | ♟ | ♟ | ♟ |
   -----------------------------------------
3  |    |    | ♞ |    |    |    |    |    |
   -----------------------------------------
4  | ♟ |    |    | ♟ |    | ♝ |    |    |
   -----------------------------------------
5  |    |    |    |    |    |    |    | ♛ |
   -----------------------------------------
6  |    | ♙ | ♘ |    |    |    |    | ♙ |
   -----------------------------------------
7  |    | ♙ | ♙ | ♙ | ♙ | ♙ | ♙ |    |
   -----------------------------------------
8  | ♖ |    | ♗ | ♕ | ♔ | ♗ | ♘ | ♖ |
   -----------------------------------------
     a    b    c    d    e    f    g    h  


In [ ]:
# check EnPassant 
chess_board = ChessBoard()
chess_board.place_piece()

# black
chess_board.move_any_pieces((1,1), (3,1))
chess_board.move_any_pieces((3,1), (4,1))
chess_board.move_any_pieces((6,0), (4,0))
#chess_board.move_any_pieces((1,7), (2,7)) # black moves another piece and lose EnPassant
chess_board.move_any_pieces((4,1), (5,0))

# white
chess_board.move_any_pieces((6,2), (4,2))
chess_board.move_any_pieces((4,2), (3,2))
chess_board.move_any_pieces((1,3), (3,3))
chess_board.move_any_pieces((3,2), (2,3))

chess_board.display()

captured pieces:  ['♟']
   -----------------------------------------
1  | ♜ | ♞ | ♝ | ♛ | ♚ | ♝ | ♞ | ♜ |
   -----------------------------------------
2  | ♟ |    | ♟ |    | ♟ | ♟ | ♟ | ♟ |
   -----------------------------------------
3  |    |    |    | ♙ |    |    |    |    |
   -----------------------------------------
4  |    |    |    |    |    |    |    |    |
   -----------------------------------------
5  |    |    |    |    |    |    |    |    |
   -----------------------------------------
6  | ♟ |    |    |    |    |    |    |    |
   -----------------------------------------
7  |    | ♙ |    | ♙ | ♙ | ♙ | ♙ | ♙ |
   -----------------------------------------
8  | ♖ | ♘ | ♗ | ♕ | ♔ | ♗ | ♘ | ♖ |
   -----------------------------------------
     a    b    c    d    e    f    g    h  
captured pieces:  ['♙'] 



In [ ]:
# check invalid EnPassant 
chess_board = ChessBoard()
chess_board.place_piece()

# black
chess_board.move_any_pieces((1,1), (3,1))
chess_board.move_any_pieces((3,1), (4,1))
chess_board.move_any_pieces((6,0), (4,0))
chess_board.move_any_pieces((4,0), (5,2))

chess_board.display()

   -----------------------------------------
1  | ♜ | ♞ | ♝ | ♛ | ♚ | ♝ | ♞ | ♜ |
   -----------------------------------------
2  | ♟ |    | ♟ | ♟ | ♟ | ♟ | ♟ | ♟ |
   -----------------------------------------
3  |    |    |    |    |    |    |    |    |
   -----------------------------------------
4  |    |    |    |    |    |    |    |    |
   -----------------------------------------
5  | ♙ | ♟ |    |    |    |    |    |    |
   -----------------------------------------
6  |    |    |    |    |    |    |    |    |
   -----------------------------------------
7  |    | ♙ | ♙ | ♙ | ♙ | ♙ | ♙ | ♙ |
   -----------------------------------------
8  | ♖ | ♘ | ♗ | ♕ | ♔ | ♗ | ♘ | ♖ |
   -----------------------------------------
     a    b    c    d    e    f    g    h  
